In [ ]:
from time import time

from wavenet.audiodata import AudioData
from wavenet.models_torch import Model

from IPython.display import Audio

%matplotlib inline

x_len = 80
y_len = 1

## Create dataset

In [ ]:
filelist = ['assets/sine.wav']
dataset = AudioData(filelist, x_len, y_len, store_tracks=True)

Audio(dataset.tracks[0]['audio'], rate=dataset.tracks[0]['sample_rate'])

## Create dataloader

## Define training parameters